Goal of this notebook is to collect and curate some history like data for various languages
```
Source 1 -> Multilingual summarization dataset)[https://www.kaggle.com/datasets/thedevastator/mlsam-multilingual-summarization-dataset]

Source 2 -> https://github.com/microsoft/msmarco/blob/master/Datasets.md
            Don't download this link (VERY HUGE) - msmarco-docs.tsv
            Have selected some 500k examples from this for exploration
```

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
from urllib.parse import urlparse
import pickle

In [2]:
# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

In [3]:
from src.constants import EMBEDDING_MODELS_DICT
from src.feature_extractor import FeatureExtractor

In [4]:
MLSAM_DATA_PATH = "/Users/cgopal/Downloads/mlsam-multilingual-summarization-dataset/"

In [5]:
!ls -ltr $MLSAM_DATA_PATH

total 9471984
-rw-rw-r--@ 1 cgopal  staff    46828273 Dec  2  2023 de_test.csv
-rw-rw-r--  1 cgopal  staff   846124338 Dec  2  2023 de_train.csv
-rw-rw-r--@ 1 cgopal  staff    47087676 Dec  2  2023 de_validation.csv
-rw-rw-r--  1 cgopal  staff    71117937 Dec  2  2023 es_test.csv
-rw-rw-r--  1 cgopal  staff  1211703726 Dec  2  2023 es_train.csv
-rw-rw-r--  1 cgopal  staff    50529088 Dec  2  2023 es_validation.csv
-rw-rw-r--@ 1 cgopal  staff    69447109 Dec  2  2023 fr_test.csv
-rw-rw-r--  1 cgopal  staff  1468931669 Dec  2  2023 fr_train.csv
-rw-rw-r--  1 cgopal  staff    70196782 Dec  2  2023 fr_validation.csv
-rw-rw-r--  1 cgopal  staff     9646755 Dec  2  2023 ru_test.csv
-rw-rw-r--@ 1 cgopal  staff   257038607 Dec  2  2023 ru_train.csv
-rw-rw-r--  1 cgopal  staff    27686809 Dec  2  2023 tu_test.csv
-rw-rw-r--@ 1 cgopal  staff     9119261 Dec  2  2023 ru_validation.csv
-rw-rw-r--  1 cgopal  staff   638760165 Dec  2  2023 tu_train.csv
-rw-rw-r--  1 cgopal  staff    25404303 Dec  2 

In [6]:
mlsam_data_list = []
for fname in tqdm(os.listdir(MLSAM_DATA_PATH)):
    file_path = os.path.join(MLSAM_DATA_PATH, fname)
    df = pd.read_csv(file_path)
    print(file_path, "->", len(df))
    df['lang'] = fname[:2]
    mlsam_data_list.append(df)
    

  7%|██████▏                                                                                      | 1/15 [00:06<01:37,  6.95s/it]

/Users/cgopal/Downloads/mlsam-multilingual-summarization-dataset/tu_train.csv -> 249277


 13%|████████████▍                                                                                | 2/15 [00:07<00:40,  3.10s/it]

/Users/cgopal/Downloads/mlsam-multilingual-summarization-dataset/es_validation.csv -> 10358


 27%|████████████████████████▊                                                                    | 4/15 [00:14<00:31,  2.87s/it]

...

/Users/cgopal/Downloads/mlsam-multilingual-summarization-dataset/de_validation.csv -> 11394


 93%|█████████████████████████████████████████████████████████████████████████████████████▊      | 14/15 [00:41<00:03,  3.90s/it]

/Users/cgopal/Downloads/mlsam-multilingual-summarization-dataset/fr_train.csv -> 392902


100%|████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:43<00:00,  2.93s/it]

/Users/cgopal/Downloads/mlsam-multilingual-summarization-dataset/ru_train.csv -> 25556


In [7]:
len(mlsam_data_list)

15

In [8]:
mlsam_data_combined = pd.concat(mlsam_data_list, axis=0)
print(len(mlsam_data_combined))
mlsam_data_combined = mlsam_data_combined.loc[mlsam_data_combined['topic']!='unknown'].reset_index(drop=True)
mlsam_data_combined = mlsam_data_combined.loc[~mlsam_data_combined['title'].isna()].reset_index(drop=True)
mlsam_data_combined['description'] = mlsam_data_combined['summary'].fillna('').apply(lambda summary: summary[:300])
print("After removal of unknown topics")
print(len(mlsam_data_combined))

1259096
After removal of unknown topics
985405


In [9]:
mlsam_data_combined['lang'].value_counts()

lang
fr    424715
es    290645
de    242982
ru     27063
Name: count, dtype: int64

In [10]:
mlsam_data_combined

,text,summary,topic,url,title,date,lang,description
0,Un niño de tres años ha fallecido esta noche e...,El niño llegó al hospital sin respiración ni p...,politica actualidad,http://elpais.com/politica/2019/01/01/actualid...,Muere un niño de tres años atragantado con una...,01/01/2019,es,El niño llegó al hospital sin respiración ni p...
1,El bikini de flores y tul de Cristina Pedroche...,La presentadora cambió su arriesgado dos pieza...,elpais gente,http://elpais.com/elpais/2019/01/01/gente/1546...,El segundo vestido de Pedroche que quedó eclip...,01/01/2019,es,La presentadora cambió su arriesgado dos pieza...
2,Una joven británica de 25 años ha resultado he...,La mujer se despeñó en el municipio de Algaida...,politica actualidad,http://elpais.com/politica/2019/01/01/actualid...,Una joven británica resulta herida muy grave a...,01/01/2019,es,La mujer se despeñó en el municipio de Algaida...
3,Juan Carlos Galindo Con este comentario de nue...,El alemán Christian Thielemann ha sido el enca...,cultura actualidad,http://elpais.com/cultura/2019/01/01/actualida...,Así te hemos contado el Concierto de Año Nuevo...,01/01/2019,es,El alemán Christian Thielemann ha sido el enca...
4,Agentes de policía detienen al atacante en Man...,Detenido un hombre tras acuchillar a tres pers...,internacional actualidad,http://elpais.com/internacional/2019/01/01/act...,La policía investiga como un ataque terrorista...,01/01/2019,es,Detenido un hombre tras acuchillar a tres pers...
...,...,...,...,...,...,...,...,...
985400,"— Георгий Евгеньевич, складывается впечатление...",Они чувствуют себя чужими в мире взрослых и тя...,social,https://www.mk.ru/social/2013/03/19/828147-kak...,Как не стать педофилом?,20/03/2013,ru,Они чувствуют себя чужими в мире взрослых и тя...
985401,"Открывая заседание, президент ассоциации Валер...",К такому выводу пришли представители малых и с...,mosobl,https://www.mk.ru/mosobl/2013/03/19/828148-ray...,Райсовет — на обочине,20/03/2013,ru,К такому выводу пришли представители малых и с...
985402,На улице Марины Цветаевой и на соседней Спарта...,Власти Подмосковья в лице врио губернатора Анд...,mosobl,https://www.mk.ru/mosobl/2013/03/19/828197-po-...,По закону против джунглей,20/03/2013,ru,Власти Подмосковья в лице врио губернатора Анд...
985403,Что не нравится жителям города? В последние го...,В Клину строительство высотки в исторической ч...,mosobl,https://www.mk.ru/mosobl/2013/03/19/828203-mes...,Местный батюшка получил место в торговых рядах,20/03/2013,ru,В Клину строительство высотки в исторической ч...


In [11]:
mlsam_data_combined['summary'].apply(len).describe(percentiles=[.25, .5,  .6,.7, .8, .9, .95, .98, .99])

count    985405.000000
mean        150.525854
std          57.657746
min          28.000000
25%         109.000000
50%         143.000000
60%         157.000000
70%         172.000000
80%         192.000000
90%         224.000000
95%         256.000000
98%         299.000000
99%         329.000000
max        1522.000000
Name: summary, dtype: float64

In [12]:
mlsam_data_combined['topic'].value_counts()

topic
sport                                            74601
politik                                          74492
economie                                         45957
wirtschaft                                       40141
idees                                            27022
                                                 ...  
reseaux-sociaux                                      1
israel-palestine                                     1
remettre-le-confort-au-centre-de-l-innovation        1
petrole                                              1
voyage-togo                                          1
Name: count, Length: 862, dtype: int64

In [13]:
mlsam_data_combined.groupby('lang')['topic'].value_counts()[:60]

lang  topic                       
de    politik                         74492
      sport                           53814
      wirtschaft                      40141
      panorama                        22922
      digital                          9811
      geld                             8359
      reise                            7510
      auto                             6742
      karriere                         6613
      muenchen                         5802
      stil                             3118
      bildung                          2754
      service                           819
      reisefuehrer                       49
      thema                              25
      app                                 6
      1.2527726?utm_source=FlexiTM        1
      1.2699904?SRCmuc=FlexiTM            1
      kultur                              1
      mahjong                             1
      tiananmen                           1
es    elpais opinion                  258

In [14]:
EN_MSMARCO_DATA_PATH = "/Users/cgopal/Downloads/web_data/msmarco_english.csv"

In [15]:
english_data = pd.read_csv(EN_MSMARCO_DATA_PATH, header=None)
english_data.columns = ['docid', 'url', 'title', 'body']
english_data['lang'] = 'en'
english_data['topic'] = ''
english_data = english_data.loc[~english_data['title'].isna()].reset_index(drop=True)
english_data['description'] = english_data['body'].fillna('').apply(lambda body: body[:300])
print(len(english_data))
english_data

495757


,docid,url,title,body,lang,topic,description
0,D1555982,https://answers.yahoo.com/question/index?qid=2...,The hot glowing surfaces of stars emit energy ...,Science & Mathematics Physics The hot glowing ...,en,,Science & Mathematics Physics The hot glowing ...
1,D301595,http://childparenting.about.com/od/physicalemo...,Developmental Milestones and Your 8-Year-Old C...,School-Age Kids Growth & Development Developme...,en,,School-Age Kids Growth & Development Developme...
2,D1359209,http://visihow.com/Check_for_Lice_Nits,Check for Lice Nits,Check for Lice Nits Edited by Mian Sheilette O...,en,,Check for Lice Nits Edited by Mian Sheilette O...
3,D2147834,http://www.nytimes.com/2010/01/05/business/glo...,Dubai Opens a Tower to Beat All,Global Business Dubai Opens a Tower to Beat Al...,en,,Global Business Dubai Opens a Tower to Beat Al...
4,D1568809,http://www.realtor.com/realestateandhomes-sear...,"Coulterville, CA Real Estate & Homes for Sale","Coulterville, CA Real Estate & Homes for Sale4...",en,,"Coulterville, CA Real Estate & Homes for Sale4..."
...,...,...,...,...,...,...,...
495752,D481294,http://biology.about.com/od/organsystems/ss/ly...,Lymphatic System Components,"Science, Tech, Math ›Science Lymphatic System ...",en,,"Science, Tech, Math ›Science Lymphatic System ..."
495753,D2920233,http://www.freshplaza.com/article/154127/Pomeg...,"Pomegranates: increasing production, consumpti...","Pomegranates: increasing production, consumpti...",en,,"Pomegranates: increasing production, consumpti..."
495754,D1616157,http://www.airpassengerrights.eu/en/denied-boa...,Denied boarding,Denied boarding3 1 1 1 1 1 1 1 1 1 1 Rating 3....,en,,Denied boarding3 1 1 1 1 1 1 1 1 1 1 Rating 3....
495755,D1363414,http://www.augustaga.gov/657/Solid-Waste,Environmental Services,Home Departments A - E Environmental Services ...,en,,Home Departments A - E Environmental Services ...


In [16]:
combined_data = pd.concat([
    mlsam_data_combined[['url', 'title', 'description', 'topic', 'lang']],
    english_data[['url', 'title', 'description', 'topic', 'lang']]
], axis=0).reset_index(drop=True)

combined_data = combined_data.loc[combined_data['title'].apply(lambda title: len(title) > 5 and len(title) < 200)].reset_index(drop=True)

combined_data['domain'] = combined_data['url'].apply(lambda x: urlparse(x).netloc.split(':')[0])
combined_data['combined_text'] = combined_data['title'] + " " +   combined_data['description'].fillna("")
print(len(combined_data))
combined_data

1423693


,url,title,description,topic,lang,domain,combined_text
0,http://elpais.com/politica/2019/01/01/actualid...,Muere un niño de tres años atragantado con una...,El niño llegó al hospital sin respiración ni p...,politica actualidad,es,elpais.com,Muere un niño de tres años atragantado con una...
1,http://elpais.com/elpais/2019/01/01/gente/1546...,El segundo vestido de Pedroche que quedó eclip...,La presentadora cambió su arriesgado dos pieza...,elpais gente,es,elpais.com,El segundo vestido de Pedroche que quedó eclip...
2,http://elpais.com/politica/2019/01/01/actualid...,Una joven británica resulta herida muy grave a...,La mujer se despeñó en el municipio de Algaida...,politica actualidad,es,elpais.com,Una joven británica resulta herida muy grave a...
3,http://elpais.com/cultura/2019/01/01/actualida...,Así te hemos contado el Concierto de Año Nuevo...,El alemán Christian Thielemann ha sido el enca...,cultura actualidad,es,elpais.com,Así te hemos contado el Concierto de Año Nuevo...
4,http://elpais.com/internacional/2019/01/01/act...,La policía investiga como un ataque terrorista...,Detenido un hombre tras acuchillar a tres pers...,internacional actualidad,es,elpais.com,La policía investiga como un ataque terrorista...
...,...,...,...,...,...,...,...
1423688,http://biology.about.com/od/organsystems/ss/ly...,Lymphatic System Components,"Science, Tech, Math ›Science Lymphatic System ...",,en,biology.about.com,"Lymphatic System Components Science, Tech, Mat..."
1423689,http://www.freshplaza.com/article/154127/Pomeg...,"Pomegranates: increasing production, consumpti...","Pomegranates: increasing production, consumpti...",,en,www.freshplaza.com,"Pomegranates: increasing production, consumpti..."
1423690,http://www.airpassengerrights.eu/en/denied-boa...,Denied boarding,Denied boarding3 1 1 1 1 1 1 1 1 1 1 Rating 3....,,en,www.airpassengerrights.eu,Denied boarding Denied boarding3 1 1 1 1 1 1 1...
1423691,http://www.augustaga.gov/657/Solid-Waste,Environmental Services,Home Departments A - E Environmental Services ...,,en,www.augustaga.gov,Environmental Services Home Departments A - E ...


In [17]:
# sample_data = combined_data.sample(200000)
# print(len(sample_data))


# sampling logic

sample_size = 50000
language_distribution = {
    'en': 0.7 * sample_size,  # 70% English
    'fr': 0.1 * sample_size,  # 10% French
    'es': 0.1 * sample_size,  # 10% Spanish
    'de': 0.1 * sample_size,  # 10% German
    'ru': 0.1 * sample_size   # 10% Russian
}

# Sample data for each language
sampled_dfs = []
for lang, size in language_distribution.items():
    sampled_dfs.append(combined_data[combined_data['lang'] == lang].sample(n=int(size), random_state=42))

# Combine the sampled data
sampled_df = pd.concat(sampled_dfs, axis=0).reset_index(drop=True)

# Display the result
print(sampled_df['lang'].value_counts())

lang
en    35000
fr     5000
es     5000
de     5000
ru     5000
Name: count, dtype: int64


In [18]:
combined_data['lang'].value_counts()

lang
en    438426
fr    424630
es    290610
de    242978
ru     27049
Name: count, dtype: int64

In [19]:
sampled_df['lang'].value_counts() / combined_data['lang'].value_counts()

lang
en    0.079831
fr    0.011775
es    0.017205
de    0.020578
ru    0.184850
Name: count, dtype: float64

In [20]:

sample_data_by_lang = {}
sample_embeddings_by_lang = {}
fe_en = FeatureExtractor(EMBEDDING_MODELS_DICT, model_name="Xenova/all-MiniLM-L6-v2")
fe_multi = FeatureExtractor(EMBEDDING_MODELS_DICT, model_name="Xenova/paraphrase-multilingual-MiniLM-L12-v2")
for lang in tqdm(sampled_df['lang'].unique()):
    print(f"preparing for lang = {lang}")
    sample_data_by_lang[lang] = sampled_df[sampled_df['lang'] == lang].reset_index(drop=True)
    if lang == "en":
        sample_embeddings_by_lang[lang] = fe_en.get_embeddings(sample_data_by_lang[lang]['combined_text'].values.tolist())
    else:
        sample_embeddings_by_lang[lang] = fe_multi.get_embeddings(sample_data_by_lang[lang]['combined_text'].values.tolist())

selected model is Xenova/all-MiniLM-L6-v2
selected model is Xenova/paraphrase-multilingual-MiniLM-L12-v2


  0%|                                                                                                      | 0/5 [00:00<?, ?it/s]

preparing for lang = en


 20%|██████████████████▌                                                                          | 1/5 [14:06<56:26, 846.66s/it]

preparing for lang = fr


...

 60%|███████████████████████████████████████████████████████▊                                     | 3/5 [17:07<08:36, 258.17s/it]

preparing for lang = de


 80%|██████████████████████████████████████████████████████████████████████████▍                  | 4/5 [18:41<03:13, 193.37s/it]

preparing for lang = ru



00%|█████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [19:59<00:00, 239.89s/it]

In [21]:
with open("../data/sample_data_by_lang.pkl", "wb") as f:
    pickle.dump(sample_data_by_lang, f)

with open("../data/sample_embeddings_by_lang.pkl", "wb") as f:
    pickle.dump(sample_embeddings_by_lang, f)